In [37]:
import json
import ollama
import markdown
from datetime import datetime

In [38]:

# Prompt
def generate_prompt(ticket):
    prompt = f"""
    You are a senior DevOps engineer.

    Generate a professional troubleshooting article in clean Markdown format.

    Sections required:
    - Issue Summary
    - Description
    - Root Cause Analysis
    - Troubleshooting & Resolution Steps (numbered)
    - Preventive Measures

    IMPORTANT:
    - Use proper Markdown formatting.
    - Use numbered lists for ordered steps.
    - Use dash (-) for bullet points.
    - Ensure proper spacing before lists.

    Ticket Data:
    {json.dumps(ticket, indent=2)}
    """
    return prompt

In [39]:
# Generate Article from Ollama
def generate_article(prompt):
    response = ollama.chat(
        model="qwen2.5:3b",
        messages=[{"role": "user", "content": prompt}]
    )

    article_markdown = response["message"]["content"]

    return article_markdown


In [ ]:
def clean_markdown(article_markdown):
    cm = article_markdown.replace("1.", "\n1.")
    cm = cm.replace("- ", "\n- ")
    return cm


In [41]:


# Convert Markdown → HTML
def markdown_to_html(clean_markdown,ticket):
    article_html_body = markdown.markdown(
        clean_markdown,
        extensions=[
            "fenced_code",
            "tables",
            "sane_lists" 
        ]
    )

    # Wrap in Professional HTML Template

    html_template = f"""
    <!DOCTYPE html>
    <html>
    <head>
    <meta charset="UTF-8">
    <title>{ticket['ticket_number']} Troubleshooting Article</title>
    <style>
    body {{
        font-family: Arial, sans-serif;
        margin: 40px;
        line-height: 1.7;
        background-color: #f9f9f9;
    }}
    .container {{
        background: white;
        padding: 40px;
        border-radius: 8px;
        box-shadow: 0px 4px 10px rgba(0,0,0,0.05);
    }}
    ul, ol {{
        margin-left: 25px;
    }}

    li {{
        margin-bottom: 8px;
    }}
    h1, h2, h3 {{
        color: #2c3e50;
    }}
    code {{
        background-color: #f4f4f4;
        padding: 4px 6px;
        border-radius: 4px;
    }}
    pre {{
        background-color: #f4f4f4;
        padding: 12px;
        overflow-x: auto;
        border-radius: 6px;
    }}
    footer {{
        margin-top: 40px;
        font-size: 12px;
        color: gray;
    }}
    </style>
    </head>
    <body>
    <div class="container">
    <h1>Troubleshooting Article – {ticket['ticket_number']}</h1>
    {article_html_body}
    <footer>
    Generated on {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
    </footer>
    </div>
    </body>
    </html>
    """

    # Save HTML File
    file_name = f"{ticket['ticket_number']}_Troubleshooting_Article.html"

    with open(file_name, "w", encoding="utf-8") as f:
        f.write(html_template)

    print(f"HTML file generated: {file_name}")

In [42]:
def main():
    # Load JSON file
    with open("ticket.json", "r") as f:
        data = json.load(f)
    
    for ticket in data:
        prompt = generate_prompt(ticket)
        article_markdown = generate_article(prompt)
        cm = clean_markdown(article_markdown)
        markdown_to_html(cm,ticket)

if __name__ == "__main__":
    main()

Cleaned Markdown:
 # Production API Server Inaccessible Due to Full Filesystem (OPS-1024)

## Issue Summary

The production API server became unreachable due to a full root filesystem. SSH login was slow, and application containers were repeatedly restarting. Investigation revealed that the root partition utilization reached 100%, which prevented services from writing logs and temporary files.

## Description

Production monitoring alerted us about an issue with disk usage at 100% on the production API server's root partition. SSH access to the server became slow, and application containers started restarting frequently. This caused significant downtime for critical services relying on this API server.

Upon further investigation, we found that a substantial amount of data was stored in log files and temporary directories, leading to the filesystem exhaustion.

## Root Cause Analysis

The root cause of the issue can be attributed to two main factors:


1. **Uncontrolled Growth of Appli